In [1]:
import time
from typing import Callable
from collections import deque
import itertools
import heapq
import random

# 自制 `Event Loop`

In [2]:
class EventLoop:
    def __init__(self):
        self._ready = deque()
        self._scheduled = []
        self._stopping = False

    def call_soon(self, callback: Callable, *args):
        self._ready.append((callback, args))

    def call_later(self, delay: float, callback: Callable, *args):
        # heap push 会按照时间顺序升序排序
        heapq.heappush(self._scheduled, (time.time() + delay, callback, args))

    def stop(self):
        self._stopping = True

    def run_once(self):
        # 处理定时任务
        now = time.time()
        if self._scheduled:
            # 如果时间到了，加到就绪队列中
            if self._scheduled[0][0] < now:
                _, callback, args = heapq.heappop(self._scheduled)
                self._ready.append((callback, args))

        # 执行就绪队列中的任务
        num = len(self._ready)
        for i in range(num):
            callback, args = self._ready.popleft()
            callback(*args)

    def run_forever(self):
        while True:
            self.run_once()
            if self._stopping:
                break

In [3]:
# 定义辅助 class, 用来包裹 yield 内容, 实现 yield from 调用原本 yield 的内容
class Awaitable:
    def __init__(self, obj):
        self.value = obj

    def __await__(self):
        yield self

In [4]:
async def small_step():
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()

    sleep_time = random.random()
    await Awaitable(sleep_time)

    t2 = time.time()
    assert t2 - t1 > sleep_time, "睡眠时间不足"
    return 123

In [5]:
async def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")

    small_result = await small_step()

    print("     end small_step")
    return small_result * 1000

In [6]:
async def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_result = await big_step()

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [7]:
task_id_counter = itertools.count(1)

In [8]:
class Task:
    def __init__(self, coro) -> None:
        self.coro = coro
        self._done = False
        self._result = None
        self._id = f"Task-{next(task_id_counter)}"

    def run(self):
        print(f"----- { self._id } starting -----")
        if not self._done:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                # 处理协程返回值
                self.result = e.value
                self._done = True
            else:
                print(f"     got {x}")
                # 上面改造 Awaitable yield self 之后，可以保证这里的 x 一定是 Awaitable 类型
                assert isinstance(x, Awaitable)
                loop.call_later(x.value, self.run)

        else:
            print("Task is done")
        print(f"----- { self._id } ending -----")


In [ ]:
loop = EventLoop()

for i in range(10):
    t = Task(one_task())
    # 添加任务到事件循环
    loop.call_soon(t.run)

# 运行 1 秒后停止
loop.call_later(1, loop.stop)

loop.run_forever()

# 根据打印可以看到开始执行时是有顺序的，不过后面执行时就按照 sleep time 顺序执行了。

----- Task-1 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C0183B470>
----- Task-1 ending -----
----- Task-2 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C0183B2F0>
----- Task-2 ending -----
----- Task-3 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C01870350>
----- Task-3 ending -----
----- Task-4 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C01870200>
----- Task-4 ending -----
----- Task-5 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C01870050>
----- Task-5 ending -----
----- Task-6 starting -----
begin task
     begin big_step
     begin small_step
     got <__main__.Awaitable object at 0x0000023C018700B0>
----- Task-6 ending -----
----